<a href="https://colab.research.google.com/github/MatRitchie/Bachelor/blob/main/Information_Retrieval.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Setup

In [1]:
!pip install allennlp

     |████████████████████████████████| 491kB 7.8MB/s 
     |████████████████████████████████| 317kB 18.4MB/s 
     |████████████████████████████████| 133kB 22.6MB/s 
     |████████████████████████████████| 266kB 21.4MB/s 
     |████████████████████████████████| 778kB 29.1MB/s 
     |████████████████████████████████| 6.7MB 68.1MB/s 
     |████████████████████████████████| 71kB 8.3MB/s 
     |████████████████████████████████| 3.0MB 76.8MB/s 
     |████████████████████████████████| 890kB 70.2MB/s 
     |████████████████████████████████| 1.1MB 51.7MB/s 
  Created wheel for overrides: filename=overrides-3.1.0-cp36-none-any.whl size=10174 sha256=1b3d6808a367256fe76c1c4e46b838a2b4f4a11933c8e43b3bca52631e1ae0d1
  Stored in directory: /root/.cache/pip/wheels/5c/24/13/6ef8600e6f147c95e595f1289a86a3cc82ed65df57582c65a9
  Created wheel for jsonnet: filename=jsonnet-0.16.0-cp36-cp36m-linux_x86_64.whl size=3321570 sha256=364b296c6852c1918e040ca568e925c5719332db4b9fc016f963c0acc2185c3e
  Stored in d

In [2]:
!pip install registrable

In [3]:
#@title Import resources
import pandas as pd
import numpy as np
import tempfile
from typing import Dict, Iterable, List, Tuple
import logging 
from overrides import overrides

from allennlp.common.file_utils import cached_path


import torch
import torchvision

import allennlp
from allennlp.data import DataLoader, DatasetReader, Instance, Vocabulary
from allennlp.data.fields import LabelField, TextField, ArrayField, ListField
from allennlp.models import Model
from allennlp.modules import TextFieldEmbedder, Seq2VecEncoder
from allennlp.modules.text_field_embedders import BasicTextFieldEmbedder
from allennlp.modules.seq2vec_encoders import BagOfEmbeddingsEncoder
from allennlp.nn import util,Activation
from allennlp.data.vocabulary import Vocabulary
from allennlp.training.metrics import CategoricalAccuracy
from allennlp.training.optimizers import AdamOptimizer
from allennlp.training.trainer import Trainer, GradientDescentTrainer
from allennlp.training.util import evaluate
from google.colab import files
from allennlp.modules.seq2vec_encoders.cls_pooler import ClsPooler
from allennlp.common import FromParams #For config files
from allennlp.common.checks import ConfigurationError
from registrable import Registrable
import os
import json
import csv

from allennlp.training.metrics.metric import Metric
from torch.utils import data
from torch.utils.data import DataLoader
from torch import IntTensor
from allennlp.data import allennlp_collate
from allennlp.modules import FeedForward, Seq2SeqEncoder, Seq2VecEncoder, TextFieldEmbedder, TimeDistributed
from allennlp.nn import InitializerApplicator, util
from allennlp.nn.util import get_text_field_mask

from allennlp.training.metrics import CategoricalAccuracy, BooleanAccuracy, Auc, F1Measure, FBetaMeasure, PearsonCorrelation

from torch import Tensor
from allennlp.common.registrable import Registrable
from allennlp.data import TextFieldTensors

from allennlp.modules.token_embedders import (
    Embedding,
    TokenCharactersEncoder,
    ElmoTokenEmbedder,
    PretrainedTransformerEmbedder,
    PretrainedTransformerMismatchedEmbedder,
)

from allennlp.data.token_indexers import (
    TokenIndexer,
    SingleIdTokenIndexer,
    TokenCharactersIndexer,
    ELMoTokenCharactersIndexer,
    PretrainedTransformerIndexer,
    PretrainedTransformerMismatchedIndexer,
)

from allennlp.data.tokenizers import (
    Token, 
    Tokenizer,
    CharacterTokenizer,
    PretrainedTransformerTokenizer,
    SpacyTokenizer,
    WhitespaceTokenizer,
)

from allennlp.modules.attention import (
    DotProductAttention,
    BilinearAttention,
    LinearAttention,
)
from allennlp.modules.matrix_attention import (
    DotProductMatrixAttention,
    BilinearMatrixAttention,
    LinearMatrixAttention,
)
from allennlp.nn import Activation


In [5]:
#@title Mounting google drive

from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
#@title Downloading kaggle database

os.environ['KAGGLE_CONFIG_DIR'] = "/content/gdrive/My Drive/Kaggle"
# /content/gdrive/My Drive/Kaggle is the path where kaggle.json is present in the Google Drive

#changing the working directory
%cd /content/gdrive/My Drive/Kaggle
!kaggle datasets download -d pratiksaha198/lyrics-generation

#unzipping the zip files and deleting the zip files
!unzip \*.zip  && rm *.zip

In [ ]:
#@title Check GPU, devices, memory

import tensorflow as tf
tf.test.gpu_device_name()

#Listing devices used
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

#Memory resources available
!cat /proc/meminfo


#Load data

Currently -> loading a dataset og artist, song title, lyrics from a kaggle datset, which is stored in my google drive

In [35]:
#Loading of Kaggle dataset

path = '/content/gdrive/My Drive/Kaggle/LYRICS_DATASET_train.csv'
data = pd.read_csv(path)
data.head()

,Artist,Title,Lyric,Procentage
0,Adam Melchor,calm forward,bugs will always bite me needles will never fr...,100.0
1,Hozier,nina cried power,its not the waking its the rising it is the g...,100.0
2,Issac Gracie,​reverie,hey babe what do you expect me to say i could...,0.0
3,Ed Sheeran,little lady,listen little lady this is just the worst way...,100.0
4,John Mayer,emoji of a wave,oh honey you dont have to try so hard to hurt...,50.0


# Making a configuration file 
This takes to construction parameters from different objects, and then put them in a dictionary in a JSON file

Motivation to keep configuration of an object seperate from the implementation code. 
Dependency injection: all of the instance variables are given a contructor parameters. 

FromParams-> constructing arguments for an object from JSON
It is neccesary to use type annotations for constructor arguments, such that it can be parsed correctly from JSON. The object is then created from a JSON dictionary instead of the constructor 

In AllenNLP all classes inherit from FromParam, such that one configuration file recursivly can be used to specify an entire experiment. 


# Defining input and output
There is an instance object, which consists of one or more fields. A field represents one piece of data, which can be either input data or output data. A field gets converted into a tensor, and fed to the model.

In the case of information retrivalm the input will be a query consisting of a song title, and the output will be the song lyric

In [7]:
#@title Default title text

#The input is song lyrics 
song_lyric: TextField

#This input will contain all of the song title eg. [title1, title2, title3...]
song_title: ListField 

#This is a one hot vector, which tells what title is correct for the lyric eg. [0, 0, 1, 0, 0]
label: ArrayField 

# Reading data
We want to read the data, and represent it with an internal structure. For that purpose we want to build a DatasetReader. We want to transform rw data into instance which match the input/output specifacations. 

Input: lyric, list of all titles, one hot vector label

output: ranking 

An instance should consist of 1 song and all titles and a label, which is a one hot vector 

In [36]:
logger = logging.getLogger(__name__)


#@DatasetReader.register("ranker")
class DatasetReaderRanker(DatasetReader):
    def __init__(
        self,
        tokenizer: Tokenizer = None,
        token_indexers: Dict[str, TokenIndexer] = None,
        max_tokens: int = None,
        **kwargs,
    ) -> None:
        super().__init__(**kwargs)
        self.tokenizer = tokenizer or WhitespaceTokenizer()
        self.token_indexers = token_indexers or {'tokens': SingleIdTokenIndexer()}
        self.max_tokens = max_tokens
        

    @overrides
    def _read(self, file_path: str):
        logger.info("Reading instances from lines in file at: %s", file_path)
        feature_names = ['artist', 'title','lyric', 'percentage']
        df = pd.read_csv(cached_path(file_path),header=None, skiprows=1, names = feature_names)            #using pandas to open file 
        df = df.dropna(thresh=df.shape[1])


        title_list = df['title'].tolist() #getting a list of all the titles ['title1', 'title2'...]
        df.drop(columns=['artist', 'title', 'percentage'], axis=1, inplace=True)      #columns, which won't be used 

        res = []
        for i in range(df.shape[0]): 
          zero_list = [0] * df.shape[0]
          zero_list[i] = 1
          res.append(zero_list)
       
        i = 0
        for row in df.to_dict(orient='records'):       #‘records’ : list like [{column -> value}, … , {column -> value}]
            yield self.text_to_instance( res[i], title_list,**row)
            i = i + 1
            
    def _make_textfield(self, text: str):
        tokens = self.tokenizer.tokenize(text)
        if self.max_tokens:
            tokens = tokens[:self.max_tokens]
        return TextField(tokens, token_indexers=self.token_indexers)

    @overrides
    def text_to_instance(
        self,
        labels: List[int],
        title_options: List[str],
        lyric: str, 
    ) -> Instance: 
        
        if labels:
            assert all(l >= 0 for l in labels)
            assert all((l == 0) for l in labels[len(title_options):])
            labels = labels[:len(title_options)]
        
        
        lyric_field = self._make_textfield(lyric)
        options_field = ListField([self._make_textfield(o) for o in title_options])
      
        fields = { 'lyric': lyric_field, 'title_options': options_field }

        if labels:
            labels = list(map(float, filter(lambda x: not pd.isnull(x), labels)))            
            fields['labels'] = ArrayField(np.array(labels), padding_value=-1)
        
        return Instance(fields)

  

# Making a model 
The model will take a bunch of instances, then it will predict the output from the input, and then it will compute loss

Taken parameters, which you want to train/chang in as constructor parameters makes alot of sense, since they will be easier to change

In [9]:
def first_nonzero(t):
    t = t.masked_fill(t != 0, 1)
    idx = torch.arange(t.size(-1), 0, -1).type_as(t)
    indices = torch.argmax(t * idx, 1, keepdim=True)
    return indices
'''
def mrr(y_pred, y_true, mask):
    # Set the largest value in the ground truth to 1, and everything else to 0.
    # If `y_true = [0.65, 0.4, 0.1]`, then `binarized_y_true=[1,0,0]`.
    binarized_y_true = y_true.ge(y_true.max(dim=-1, keepdim=True).values).long()  #det her er nok ikke nødvendigt, da det er en one-hot vector
    y_pred = y_pred.masked_fill(~mask, -1)

    # Sort the predictions to get the predicted ranking.
    _, rank = y_pred.sort(descending=True, dim=-1)

    # Sort the ground truth labels by their predicted rank.
    # If `y_pred=[0.25, 0.55, 0.05]` and `binarized_y_true=[1, 0, 0]`,
    # then `rank=[1,0,2]` and `ordered_truth=`[0,1,0]`.
    ordered_truth = binarized_y_true.gather(-1, rank)

    # Ordered indices: [1, 2, ..., batch_size]
    print('y_pred',y_pred.shape)
    print('binarized_y_true', binarized_y_true.shape)

    ab = torch.arange(1, binarized_y_true.size(-1))
    cd = torch.arange(1, binarized_y_true.size(-1)+1)
    print('ab', ab.shape)
    print('cd', cd.shape)
    #indices = torch.arange(1, binarized_y_true.size(-1) + 1).view_as(y_pred).type_as(y_pred)

    indices = torch.arange(1, binarized_y_true.size(-1) + 1).view_as(y_pred).type_as(y_pred)

    # Calculate the reciprocal rank for each position, 0ing-out masked values.
    # Following above example, `_mrr = [0/1, 1/2 ,0/3] = [0, 0.5, 0]`.
    #_mrr = (ordered_truth / indices) * mask
    _mrr = np.true_divide(ordered_truth, indices) * mask

    # Get the mrr at the first non-zero position for each instance in the batch, and take the mean.
    # Following above example, `first_nonzero(ordered_truth) = 1`, so we grab `mrr[1] = torch.tensor(0.5)`.
    # The example only has one instance, but this works for batched input also.
    return _mrr.gather(-1, first_nonzero(ordered_truth)).mean()
'''
def mrr(y_pred, y_true, mask):
    y_pred = y_pred.masked_fill(~mask, -1)
    y_true = y_true.ge(y_true.max(dim=-1, keepdim=True).values).float()

    _, idx = y_pred.sort(descending=True, dim=-1)
    ordered_truth = y_true.gather(1, idx)
    
    gold = torch.arange(y_true.size(-1)).unsqueeze(0).type_as(y_true)
    _mrr = (ordered_truth / (gold + 1)) * mask
    
    return _mrr.gather(1, first_nonzero(ordered_truth)).mean()

class RankingMetric(Metric):
    def __init__(
        self,
        padding_value: int = -1
    ) -> None:
        self._padding_value = padding_value
        self.reset()
        
    def __call__(
            self,
            predictions: torch.LongTensor,
            gold_labels: torch.LongTensor,
            mask: torch.LongTensor = None
        ):
        """
        Parameters
        ----------
        predictions : ``torch.Tensor``, required.
            A tensor of real-valued predictions of shape (batch_size, slate_length).
        gold_labels : ``torch.Tensor``, required.
            A tensor of real-valued labels of shape (batch_size, slate_length).
        """
        
        if mask is None:
            mask = torch.ones_like(gold_labels).bool()
        
        self._all_predictions.append(predictions.detach().cpu())
        self._all_gold_labels.append(gold_labels.detach().cpu()) 
        self._all_masks.append(mask.detach().cpu())
        
    @property
    def predictions(self):
        return torch.cat(self._all_predictions, dim=0)
    
    @property
    def gold_labels(self):
        return torch.cat(self._all_gold_labels, dim=0)
    
    @property
    def masks(self):
        return torch.cat(self._all_masks, dim=0)
        
    def get_metric(self, reset: bool = False):
        raise NotImplementedError()
    
    def reset(self):
        self._all_predictions = []
        self._all_gold_labels = []
        self._all_masks = []
    
#@Metric.register("mrr")
class MRR(RankingMetric):
    def get_metric(self, reset: bool = False):
        predictions = torch.cat(self._all_predictions, dim=0)
        labels = torch.cat(self._all_gold_labels, dim=0)
        masks = torch.cat(self._all_masks, dim=0)

        score = mrr(predictions, labels, masks).item()

        if reset:
            self.reset()
        return score

In [10]:
class RelevanceMatcher(Registrable, torch.nn.Module):
    def __init__(
        self,
        input_dim: int
    ):
        super().__init__()
        self.dense = torch.nn.Linear(input_dim, 1, bias=False)


    def forward(
        self, 
        query_embeddings: TextFieldTensors, 
        candidates_embeddings: TextFieldTensors,
        query_mask: torch.Tensor = None,
        candidates_mask: torch.Tensor = None
    ):
        raise NotImplementedError()

#@RelevanceMatcher.register('bert_cls')
class BertCLS(RelevanceMatcher):
    def __init__(
        self,
        input_dim: int,
        **kwargs
    ):
        super().__init__(input_dim=input_dim*4, **kwargs)
      
        # Gets the [CLS] token from BERT
        self._seq2vec_encoder = ClsPooler(embedding_dim=input_dim)

    def forward(
        self,
        lyric_embeddings: torch.Tensor,
        option_embeddings: torch.Tensor,
        lyric_mask: torch.Tensor = None,
        option_mask: torch.Tensor = None
    )-> torch.Tensor:

        '''
        lyric_embeddings.data.shape = [batch*num_options, words, dim]
        options_embeddings.data.shape = [batch*num_options, words, dim]
        lyric_mask.size = [batch*num_options, words]
        option_mask.size = [batch*num_options, words]
        '''

        lyric_encoded = self._seq2vec_encoder(lyric_embeddings, lyric_mask)
        option_encoded = self._seq2vec_encoder(option_embeddings, option_mask)

        interaction_vector = torch.cat(
            [
                lyric_encoded,
                option_encoded,
                torch.abs(lyric_encoded-option_encoded),
                lyric_encoded*option_encoded
            ],
            dim=1,
        )
        dense_out = self.dense(interaction_vector)
        score = torch.squeeze(dense_out,1)

        return score


#@Model.register("ranker")
class DocumentRanker(Model):
    def __init__(
        self,
        vocab: Vocabulary,
        text_field_embedder: TextFieldEmbedder,
        relevance_matcher: RelevanceMatcher,
        dropout: float = None,
        num_labels: int = None,
        initializer: InitializerApplicator = InitializerApplicator(),
        **kwargs,
    ) -> None:

        super().__init__(vocab, **kwargs)
        self._text_field_embedder = text_field_embedder
        self._relevance_matcher = TimeDistributed(relevance_matcher) 
        self._mrr = MRR(padding_value=-1)
        self._loss = torch.nn.MSELoss(reduction='none')
        initializer(self)
        
    # @torchsnooper.snoop()
    def forward(  # type: ignore
        self, 
        lyric: TextFieldTensors,               # batch * words
        title_options: TextFieldTensors,       # batch * num_options * words
        labels: torch.IntTensor = None         # batch * num_options
    ) -> Dict[str, torch.Tensor]:
      
        embedded_text = self._text_field_embedder(lyric, num_wrapping_dims=0)
        mask = get_text_field_mask(lyric).long()

        embedded_options = self._text_field_embedder(title_options, num_wrapping_dims=1) 
        options_mask = get_text_field_mask(title_options, num_wrapping_dims=1).long() #har til føjet det med num_wrapping_dim

        #make pairs of (lyric, title) instead of (lyric, (title1, title2,...))
        embedded_text = embedded_text.unsqueeze(1).expand(-1, embedded_options.size(1), -1, -1)
        mask = mask.unsqueeze(1).expand(-1, embedded_options.size(1), -1)
        
        '''
        embedded_text.data.shape = [batch, num_options, words, dim]
        embedded_options.data.shape = [batch, num_options, words, dim]
        mask.size = [batch, num_options, words]
        options_mask.size = [batch, num_options, words]
        '''
        scores = self._relevance_matcher(embedded_text, embedded_options, mask, options_mask).squeeze(-1)
        probs = torch.sigmoid(scores)

        output_dict = {"logits": scores, "probs": probs}
        output_dict["token_ids"] = util.get_token_ids_from_text_field_tensors(lyric)
        
        if labels is not None:
            label_mask = (labels != -1)
            self._mrr(probs, labels, label_mask)
            probs = probs.view(-1)
            labels = labels.view(-1)
            label_mask = label_mask.view(-1)   
            loss = self._loss(probs, labels)
            output_dict["loss"] = loss.masked_fill(~label_mask, 0).sum() / label_mask.sum()

        return output_dict

    #@overrides
    def make_output_human_readable(
        self, output_dict: Dict[str, torch.Tensor]
    ) -> Dict[str, torch.Tensor]:
        return output_dict

    def get_metrics(self, reset: bool = False) -> Dict[str, float]:
        metrics = {
            "mrr": self._mrr.get_metric(reset)  
        }
        return metrics

    #default_predictor = "document_ranker"

In [15]:
#@title builds 
def build_dataset_reader() -> DatasetReader:
    return DatasetReaderRanker()

def read_data(reader: DatasetReader) -> Tuple[Iterable[Instance], Iterable[Instance]]:
    print("Reading data")
    training_data = reader.read('/content/gdrive/My Drive/Kaggle/LYRICS_DATASET_train.csv')
    validation_data = reader.read('/content/gdrive/My Drive/Kaggle/LYRICS_DATASET_dev.csv') 
    return training_data , validation_data

def build_vocab(instances: Iterable[Instance]) -> Vocabulary:
    print("Building the vocabulary")
    return Vocabulary.from_instances(instances)

def build_model(vocab: Vocabulary) -> Model:
    print("Building the model")
    vocab_size = vocab.get_vocab_size("tokens")
    embedder = BasicTextFieldEmbedder(
        {"tokens": Embedding(embedding_dim=10, num_embeddings=vocab_size)})
    encoder = BagOfEmbeddingsEncoder(embedding_dim=10)
    relevanceMatcher = BertCLS(input_dim=10)
    return DocumentRanker(vocab, embedder,relevanceMatcher)

def build_data_loaders(
    train_data: torch.utils.data.Dataset,
    dev_data: torch.utils.data.Dataset) -> Tuple[allennlp.data.DataLoader, allennlp.data.DataLoader]:
    train_loader = DataLoader(train_data, batch_size=8, shuffle=True, collate_fn=allennlp_collate)
    dev_loader = DataLoader(dev_data, batch_size=8, shuffle=False, collate_fn=allennlp_collate)
    return train_loader, dev_loader

def build_trainer(
    model: Model,
    serialization_dir: str,
    train_loader: DataLoader,
    dev_loader: DataLoader) -> Trainer:
    parameters = [
        [n, p]
        for n, p in model.named_parameters() if p.requires_grad
    ]
    optimizer = AdamOptimizer(parameters)
    trainer = GradientDescentTrainer(
        model=model,
        serialization_dir=serialization_dir,
        data_loader=train_loader,
        validation_data_loader=dev_loader,
        num_epochs=5,
        optimizer=optimizer,
    )
    return trainer


In [37]:
#@title Training loop 
def run_training_loop():
    dataset_reader = build_dataset_reader()

    train_data, dev_data = read_data(dataset_reader)

    vocab = build_vocab(train_data + dev_data)
    model = build_model(vocab)

    # This is the allennlp-specific functionality in the Dataset object;
    # we need to be able convert strings in the data to integers, and this
    # is how we do it.
    train_data.index_with(vocab)
    dev_data.index_with(vocab)

    train_loader, dev_loader = build_data_loaders(train_data, dev_data)

    # You obviously won't want to create a temporary file for your training
    # results, but for execution in binder for this guide, we need to do this.
    with tempfile.TemporaryDirectory() as serialization_dir:
        trainer = build_trainer(
            model,
            serialization_dir,
            train_loader,
            dev_loader
        )
        print("Starting training")
        trainer.train()
        print("Finished training")

    return model, dataset_reader

run_training_loop()



reading instances: 0it [00:00, ?it/s]

Reading data




reading instances: 113it [19:22, 10.28s/it]
  0%|          | 0/26 [10:01<?, ?it/s]

reading instances: 53it [00:00, 172.44it/s]
reading instances: 126it [00:00, 223.58it/s]
reading instances: 206it [00:00, 302.43it/s]
reading instances: 70it [00:00, 1140.54it/s]
building vocab: 100%|##########| 276/276 [00:00<00:00, 2149.84it/s]
You provided a validation dataset but patience was set to None, meaning that early stopping is disabled
unable to check gpu_memory_mb() due to occasional failure, continuing
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/allennlp/common/util.py", line 415, in gpu_memory_mb
    encoding="utf-8",
  File "/usr/lib/python3.6/subprocess.py", line 356, in check_output
    **kwargs).stdout
  File "/usr/lib/python3.6/subprocess.py", line 438, in run
    output=stdout, stderr=stderr)
subprocess.CalledProcessError: Command '['nvidia-smi', '--query-gpu=memory.used', '--format=csv,nounits,noheader']' returned non-zero exit status 9.
  0

Building the vocabulary
Building the model
Starting training


mrr: 0.0432, batch_loss: 0.2417, loss: 0.2456 ||: 100%|##########| 26/26 [00:02<00:00, 10.74it/s]
mrr: 0.1032, batch_loss: 0.2452, loss: 0.2448 ||: 100%|##########| 9/9 [00:00<00:00, 72.81it/s]
unable to check gpu_memory_mb() due to occasional failure, continuing
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/allennlp/common/util.py", line 415, in gpu_memory_mb
    encoding="utf-8",
  File "/usr/lib/python3.6/subprocess.py", line 356, in check_output
    **kwargs).stdout
  File "/usr/lib/python3.6/subprocess.py", line 438, in run
    output=stdout, stderr=stderr)
subprocess.CalledProcessError: Command '['nvidia-smi', '--query-gpu=memory.used', '--format=csv,nounits,noheader']' returned non-zero exit status 9.
mrr: 0.0834, batch_loss: 0.2256, loss: 0.2342 ||: 100%|##########| 26/26 [00:01<00:00, 13.15it/s]
mrr: 0.1084, batch_loss: 0.2390, loss: 0.2379 ||: 100%|##########| 9/9 [00:00<00:00, 95.91it/s]
unable to check gpu_memory_mb() due to occasional fa

Finished training


(DocumentRanker(
   (_text_field_embedder): BasicTextFieldEmbedder(
     (token_embedder_tokens): Embedding()
   )
   (_relevance_matcher): TimeDistributed(
     (_module): BertCLS(
       (dense): Linear(in_features=40, out_features=1, bias=False)
       (_seq2vec_encoder): ClsPooler()
     )
   )
   (_loss): MSELoss()
 ), <__main__.DatasetReaderRanker at 0x7ff9cc04b208>)

# Evaluation of the model 
Computation of evaluation metrics against train set 

In [38]:
model, dataset_reader = run_training_loop()

# Now we can evaluate the model on a new dataset.
test_data = dataset_reader.read('/content/gdrive/My Drive/Kaggle/LYRICS_DATASET_test.csv')
test_data.index_with(model.vocab)
data_loader = DataLoader(test_data, batch_size=8, collate_fn=allennlp_collate)

results = evaluate(model, data_loader)


reading instances: 0it [00:00, ?it/s]

Reading data


reading instances: 206it [00:00, 282.54it/s]
reading instances: 70it [00:00, 306.57it/s]
building vocab: 100%|##########| 276/276 [00:00<00:00, 2217.13it/s]
You provided a validation dataset but patience was set to None, meaning that early stopping is disabled
unable to check gpu_memory_mb() due to occasional failure, continuing
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/allennlp/common/util.py", line 415, in gpu_memory_mb
    encoding="utf-8",
  File "/usr/lib/python3.6/subprocess.py", line 356, in check_output
    **kwargs).stdout
  File "/usr/lib/python3.6/subprocess.py", line 438, in run
    output=stdout, stderr=stderr)
subprocess.CalledProcessError: Command '['nvidia-smi', '--query-gpu=memory.used', '--format=csv,nounits,noheader']' returned non-zero exit status 9.
  0%|          | 0/26 [00:00<?, ?it/s]

Building the vocabulary
Building the model
Starting training


mrr: 0.0299, batch_loss: 0.2429, loss: 0.2481 ||: 100%|##########| 26/26 [00:02<00:00, 12.02it/s]
mrr: 0.0844, batch_loss: 0.2472, loss: 0.2469 ||: 100%|##########| 9/9 [00:00<00:00, 71.04it/s]
unable to check gpu_memory_mb() due to occasional failure, continuing
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/allennlp/common/util.py", line 415, in gpu_memory_mb
    encoding="utf-8",
  File "/usr/lib/python3.6/subprocess.py", line 356, in check_output
    **kwargs).stdout
  File "/usr/lib/python3.6/subprocess.py", line 438, in run
    output=stdout, stderr=stderr)
subprocess.CalledProcessError: Command '['nvidia-smi', '--query-gpu=memory.used', '--format=csv,nounits,noheader']' returned non-zero exit status 9.
mrr: 0.0413, batch_loss: 0.2327, loss: 0.2385 ||: 100%|##########| 26/26 [00:01<00:00, 13.16it/s]
mrr: 0.0886, batch_loss: 0.2413, loss: 0.2407 ||: 100%|##########| 9/9 [00:00<00:00, 95.99it/s]
unable to check gpu_memory_mb() due to occasional fa

Finished training


reading instances: 70it [00:00, 88.25it/s]
mrr: 0.09, loss: 0.20 ||: 100%|##########| 9/9 [00:00<00:00, 74.54it/s]


#Train the model 

This is for training with commands, when using a configuration file

In [ ]:
#Just tring out different things 

#path to training configurations: /content/gdrive/My\ Drive/Bachelor_Project/training_config
!allennlp train /content/gdrive/My\ Drive/Bachelor_Project/training_config/retrieval_config.jsonnet -s /content/gdrive/My\ Drive/Bachelor_Project/Output 

In [ ]:
# Shape: (batch_size, num_tokens, embedding_dim)
        #embedded_title = self.embedder(title)  
        embedded_lyric = self.embedder(lyric)

        # Shape: (batch_size, num_tokens)
        #mask_title = util.get_text_field_mask(title)
        mask_lyric = util.get_text_field_mask(lyric)


        # Shape: (batch_size, encoding_dim)
        #encoded_title = self.encoder(embedded_title, mask_title) #We now have a single vector for each batch of instances
        encoded_lyric = self.encoder(embedded_lyric, mask_lyric)

        # Shape: (batch_size, num_labels)
        logits = self.classifier(encoded_lyric)
        # Shape: (batch_size, num_labels)
        probs = torch.nn.functional.softmax(logits)  #Get a probability distribution over labels 
        # Shape: (1,)
        loss = torch.nn.functional.cross_entropy(logits, title)  #Cross entrophy as loss function 
        self.accuracy(logits, title)

        return {'loss': loss, 'probs': probs}